### Dependencies

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("/Users/omarafifi/Downloads/STATS 503/Kaggle/Data/train.csv.zip")
X = data.drop(['y'], axis = 1)
y = data['y']

X.head()

,SEQN,self_eval,teacher_eval,extracurricular,district,SRP_1,SRP_2,SRP_3,SRP_4,SRP_5,...,SRP_41,SRP_42,SRP_43,SRP_44,SRP_45,SRP_46,SRP_47,SRP_48,SRP_49,SRP_50
0,969167,4,5,9,3,-0.181,-0.379,-0.164,0.080,0.378,...,2.070,-1.156,-0.730,-0.508,-0.497,0.224,0.412,-0.517,0.099,0.114
1,188942,4,3,5,4,-0.126,1.603,1.021,0.489,-1.404,...,1.478,-0.318,1.240,-1.993,2.021,-1.078,-0.277,0.802,0.253,-0.720
2,134058,1,2,8,5,0.724,-0.702,2.249,0.910,0.330,...,0.119,0.449,1.980,-0.401,-0.544,-0.944,1.592,0.875,-0.734,-2.336
3,124022,3,3,10,6,0.706,-0.302,1.023,-0.895,0.625,...,1.249,2.025,-2.289,-0.407,0.025,-0.515,0.408,1.380,-1.075,-2.451
4,685285,5,5,1,5,-0.350,-1.001,0.931,0.192,0.491,...,0.341,-0.118,-0.288,0.457,-0.566,0.822,-0.317,0.661,2.096,0.004


In [5]:
def process_data(df_X):
   from sklearn.preprocessing import OneHotEncoder
   import copy

   X = df_X.copy()
   
   #one hot encode districts
   for i in np.sort(X['district'].unique()):
      X[f"district {i}"] = (X['district'] == i).astype(int)
   
   X = X.drop(['district'], axis = 1)

   #this seems to be a good feature
   X['aggregate_eval'] = X[['self_eval', 'teacher_eval', 'extracurricular']].sum(axis = 1)


   X['aggregate_score'] = X[['SRP_1',
      'SRP_2', 'SRP_3', 'SRP_4', 'SRP_5', 'SRP_6', 'SRP_7', 'SRP_8', 'SRP_9',
      'SRP_10', 'SRP_11', 'SRP_12', 'SRP_13', 'SRP_14', 'SRP_15', 'SRP_16',
      'SRP_17', 'SRP_18', 'SRP_19', 'SRP_20', 'SRP_21', 'SRP_22', 'SRP_23',
      'SRP_24', 'SRP_25', 'SRP_26', 'SRP_27', 'SRP_28', 'SRP_29', 'SRP_30',
      'SRP_31', 'SRP_32', 'SRP_33', 'SRP_34', 'SRP_35', 'SRP_36', 'SRP_37',
      'SRP_38', 'SRP_39', 'SRP_40', 'SRP_41', 'SRP_42', 'SRP_43', 'SRP_44',
      'SRP_45', 'SRP_46', 'SRP_47', 'SRP_48', 'SRP_49', 'SRP_50']].sum( axis = 1)
   
   X = X.drop([
      'SRP_2', 'SRP_3', 'SRP_4', 'SRP_5', 'SRP_6', 'SRP_7', 'SRP_8', 'SRP_9',
      'SRP_10', 'SRP_11', 'SRP_12', 'SRP_13', 'SRP_14', 'SRP_15', 'SRP_16',
      'SRP_17', 'SRP_18', 'SRP_19', 'SRP_20', 'SRP_21', 'SRP_22', 'SRP_23',
      'SRP_24', 'SRP_25', 'SRP_26', 'SRP_27', 'SRP_28', 'SRP_29', 'SRP_30',
      'SRP_31', 'SRP_32', 'SRP_33', 'SRP_34', 'SRP_35', 'SRP_36', 'SRP_37',
      'SRP_38', 'SRP_39', 'SRP_40', 'SRP_41', 'SRP_42', 'SRP_43', 'SRP_44',
      'SRP_45', 'SRP_46', 'SRP_47', 'SRP_48', 'SRP_49', 'SRP_50'], axis = 1)
   
   X['aggregate_score_abs'] =  X['aggregate_score']+X['aggregate_eval']+X['SRP_1']
   
   X = X.drop(['SEQN'], axis = 1)

   return X



In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(process_data(X),y, random_state=68)
X_train.head()

,self_eval,teacher_eval,extracurricular,SRP_1,district 1,district 2,district 3,district 4,district 5,district 6,district 7,aggregate_eval,aggregate_score,aggregate_score_abs
7003,5,5,1,-0.837,0,0,1,0,0,0,0,11,-2.529,7.634
4800,2,1,8,0.162,0,0,0,0,1,0,0,11,-0.312,10.850
1816,5,5,2,1.069,0,0,0,1,0,0,0,12,10.342,23.411
362,4,3,7,0.292,0,0,0,0,0,0,1,14,4.105,18.397
7458,2,1,7,-1.573,0,1,0,0,0,0,0,10,-2.690,5.737


### Boosting

### Train the Dense Networks

grid search for hyperparamaters

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import BaggingRegressor


pipe = Pipeline( steps = 
    [
     ('poly', PolynomialFeatures(degree = 1)),
      ('scaler', StandardScaler()),
      ('classifier', MLPRegressor(activation = 'logistic',
                                  max_iter = 3000,
                                  hidden_layer_sizes = 5,
                                  solver = 'lbfgs'
                                  ))
     ]
  )


gridcv_ind = GridSearchCV(pipe, param_grid = {
                                          'classifier': [MLPRegressor(activation = 'logistic',
                                                                      max_iter = 5000,
                                                                      hidden_layer_sizes = 64,
                                                                      solver = 'lbfgs'
                                                                      )],
                                          'scaler': [StandardScaler()]
                                          },
                                          cv = 5,
                                          verbose = 3,
                                          return_train_score = True, 
                                          scoring= 'r2')

#fit the models
gridcv_ind.fit(X_train, y_train)


In [ ]:
gridcv_ind.best_params_

{'classifier': MLPRegressor(activation='logistic', hidden_layer_sizes=5, max_iter=4000,
              solver='lbfgs'),
 'scaler': StandardScaler()}

### Bag the Networks

Bag with optimized hyperparamaters to aleviate some variance:

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import BaggingRegressor


bagging_pipe = Pipeline( steps = 
    [
      ('poly', PolynomialFeatures(degree = 1)),
      ('scaler', StandardScaler()),
      ('classifier', BaggingRegressor())
     ]
  )

#Select best model with cross validation (I removed some paramater options once I finished tuning so the notebook runs faster)
gridcv = GridSearchCV(bagging_pipe, param_grid = {
                                          'classifier__estimator': [MLPRegressor(activation = 'logistic',
                                                                      max_iter = 10000,
                                                                      hidden_layer_sizes = 64,
                                                                      solver = 'lbfgs'
                                                                      )],
                                          'classifier__n_estimators': [30],
                                          'scaler': [StandardScaler()],
                                          'poly__degree': [1],
                                          },
                                          cv = 5,
                                          verbose = 3,
                                          return_train_score = True, 
                                          scoring= 'r2')

#fit the models
gridcv.fit(X_train, y_train)


In [ ]:
best_model = gridcv.best_estimator_
best_model.fit(process_data(X),y)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1)),
                ('scaler', StandardScaler()),
                ('classifier',
                 BaggingRegressor(estimator=MLPRegressor(activation='logistic',
                                                         hidden_layer_sizes=5,
                                                         max_iter=10000,
                                                         solver='lbfgs')))])

In [ ]:
test_data = pd.read_csv('/Users/omarafifi/Downloads/STATS 503/Kaggle/Data/test.csv.zip')
X_test = process_data(test_data)
X_test.head()

,self_eval,teacher_eval,extracurricular,SRP_1,district 1,district 2,district 3,district 4,district 5,district 6,district 7,aggregate_eval,aggregate_score
0,1,1,5,-0.873,0,0,1,0,0,0,0,7,-7.346
1,3,2,9,-0.948,0,0,0,0,0,1,0,14,-0.900
2,1,1,1,-0.356,1,0,0,0,0,0,0,3,7.884
3,2,2,2,-0.135,1,0,0,0,0,0,0,6,0.130
4,4,4,8,-0.955,0,0,0,0,0,1,0,16,3.339


### Export Predictions

In [ ]:
preds = best_model.predict(X_test)
output = pd.DataFrame()
output['SEQN'] = test_data['SEQN']
output['y'] = preds
output.to_csv('/Users/omarafifi/Downloads/STATS 503/Kaggle/Submission'+'/predictions.csv', index = False)